In [115]:
import pandas as pd
import numpy as np
file1 =pd.read_csv('accidents_2005_to_2007.csv', dtype='unicode')
file2 =pd.read_csv('accidents_2009_to_2011.csv', dtype='unicode')
file3 =pd.read_csv('accidents_2012_to_2014.csv', dtype='unicode')
df = df_combined = pd.concat([file1, file2, file3],ignore_index=True)
df.head(5)


,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Year,Unnamed: 0
0,200501BS00001,525680,178240,-0.19117,51.489096,1,2,1,1,04/01/2005,...,Daylight: Street light present,Raining without high winds,Wet/Damp,NaN,NaN,1,Yes,E01002849,2005,NaN
1,200501BS00002,524170,181650,-0.211708,51.520075,1,3,1,1,05/01/2005,...,Darkness: Street lights present and lit,Fine without high winds,Dry,NaN,NaN,1,Yes,E01002909,2005,NaN
2,200501BS00003,524520,182240,-0.206458,51.525301,1,3,2,1,06/01/2005,...,Darkness: Street lights present and lit,Fine without high winds,Dry,NaN,NaN,1,Yes,E01002857,2005,NaN
3,200501BS00004,526900,177530,-0.173862,51.482442,1,3,1,1,07/01/2005,...,Daylight: Street light present,Fine without high winds,Dry,NaN,NaN,1,Yes,E01002840,2005,NaN
4,200501BS00005,528060,179040,-0.156618,51.495752,1,3,1,1,10/01/2005,...,Darkness: Street lighting unknown,Fine without high winds,Wet/Damp,NaN,NaN,1,Yes,E01002863,2005,NaN


In [116]:
df.shape

(1504150, 34)

In [117]:
df.columns

Index(['Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR',
       'Longitude', 'Latitude', 'Police_Force', 'Accident_Severity',
       'Number_of_Vehicles', 'Number_of_Casualties', 'Date', 'Day_of_Week',
       'Time', 'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident',
       'LSOA_of_Accident_Location', 'Year', 'Unnamed: 0'],
      dtype='object')

In [118]:
Special_And_Carriageway_not_na=df[df['Special_Conditions_at_Site'].notna() | df['Carriageway_Hazards'].notna()]
Special_And_Carriageway_not_na.shape

(61383, 34)

In [119]:
df_filtered = df[~df.index.isin(Special_And_Carriageway_not_na.index)]
df_filtered.shape

(1442767, 34)

In [120]:
df_filtered=df.drop(columns=['Accident_Index','Longitude','Latitude','Police_Force','Did_Police_Officer_Attend_Scene_of_Accident','LSOA_of_Accident_Location', 'Local_Authority_(District)',
       'Local_Authority_(Highway)', '1st_Road_Class', '1st_Road_Number','2nd_Road_Class', '2nd_Road_Number','Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities'],axis=1)
df_filtered.columns

Index(['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Accident_Severity',
       'Number_of_Vehicles', 'Number_of_Casualties', 'Date', 'Day_of_Week',
       'Time', 'Road_Type', 'Speed_limit', 'Junction_Detail',
       'Junction_Control', 'Light_Conditions', 'Weather_Conditions',
       'Road_Surface_Conditions', 'Urban_or_Rural_Area', 'Year', 'Unnamed: 0'],
      dtype='object')

In [121]:
df_filtered = df_filtered.dropna(subset=['Time'])
df_filtered = df_filtered[df_filtered['Time'].str.match(r'^\d{1,2}:\d{2}$')]
df_filtered['Hour'] = df_filtered['Time'].str.split(':').str[0].astype(int)
df_filtered['Day_of_Week'] = pd.to_numeric(df_filtered['Day_of_Week'], errors='coerce')

print("Unique values in 'Day_of_Week':", df_filtered['Day_of_Week'].unique())
print("Unique values in 'Hour':", df_filtered['Hour'].unique())

rush_hours = [7, 8, 9, 16, 17, 18]
workdays = [1, 2, 3, 4, 5]
df_filtered['Rush_Hour'] = df_filtered.apply(lambda row: 1 if row['Hour'] in rush_hours and row['Day_of_Week'] in workdays else 0, axis=1)

print("Rush Hour distribution:")
print(df_filtered['Rush_Hour'].value_counts())

df_filtered.head(5)
print(df_filtered['Day_of_Week'].unique())
df_filtered.head(5)


Unique values in 'Day_of_Week': [3 4 5 6 2 7 1]
Unique values in 'Hour': [17  0 10 21 12 20 22 16  5 11  9  3  7 13 18 15 23  8 14 19  1  4  2  6]
Rush Hour distribution:
Rush_Hour
0    1054537
1     449496
Name: count, dtype: int64
[3 4 5 6 2 7 1]


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,Junction_Detail,Junction_Control,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour
0,525680,178240,2,1,1,04/01/2005,3,17:42,Single carriageway,30,NaN,NaN,Daylight: Street light present,Raining without high winds,Wet/Damp,1,2005,NaN,17,1
1,524170,181650,3,1,1,05/01/2005,4,17:36,Dual carriageway,30,NaN,Automatic traffic signal,Darkness: Street lights present and lit,Fine without high winds,Dry,1,2005,NaN,17,1
2,524520,182240,3,2,1,06/01/2005,5,00:15,Single carriageway,30,NaN,NaN,Darkness: Street lights present and lit,Fine without high winds,Dry,1,2005,NaN,0,0
3,526900,177530,3,1,1,07/01/2005,6,10:35,Single carriageway,30,NaN,NaN,Daylight: Street light present,Fine without high winds,Dry,1,2005,NaN,10,0
4,528060,179040,3,1,1,10/01/2005,2,21:13,Single carriageway,30,NaN,NaN,Darkness: Street lighting unknown,Fine without high winds,Wet/Damp,1,2005,NaN,21,0


In [122]:
def classify_speed_limit(speed):
    if speed in ['70', '60', '50']:  # Highway or Fast
        return 'Highway' if speed != '50' else 'Fast'
    else:  # Non-Highway or Slow
        return 'Non-Highway' if speed != '30' else 'Slow'
df_filtered['Speed_Classification'] = df_filtered['Speed_limit'].apply(classify_speed_limit)

#print(df_filtered[['Speed_limit', 'Speed_Classification']])
df_filtered.head(5)
df_filtered['Speed_Classification'].unique

<bound method Series.unique of 0                 Slow
1                 Slow
2                 Slow
3                 Slow
4                 Slow
              ...     
1504145        Highway
1504146        Highway
1504147    Non-Highway
1504148        Highway
1504149        Highway
Name: Speed_Classification, Length: 1504033, dtype: object>

In [123]:
df_filtered['Month'] = df_filtered['Date'].str.split('/').str[1].astype(int)
df_filtered.tail(5)

,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month
1504145,310037,597647,2,2,1,07/12/2014,1,22:20,Dual carriageway,70,...,Darkeness: No street lighting,Snowing without high winds,Snow,2,2014,464692,22,0,Highway,12
1504146,321509,574063,3,2,2,11/12/2014,5,06:40,Dual carriageway,70,...,Darkeness: No street lighting,Fine without high winds,Snow,2,2014,464693,6,0,Highway,12
1504147,321337,566365,3,1,1,09/12/2014,3,02:00,Single carriageway,40,...,Darkness: Street lights present and lit,Fine without high winds,Frost/Ice,2,2014,464694,2,0,Non-Highway,12
1504148,323869,566853,2,3,2,17/12/2014,4,06:55,Single carriageway,60,...,Darkeness: No street lighting,Raining without high winds,Wet/Damp,2,2014,464695,6,0,Highway,12
1504149,314072,579971,3,2,2,24/12/2014,4,15:00,Single carriageway,60,...,Daylight: Street light present,Fine without high winds,Wet/Damp,2,2014,464696,15,0,Highway,12


In [124]:
df_filtered = df_filtered.dropna(subset=['Time'])
df_filtered = df_filtered[df_filtered['Time'].str.match(r'^\d{1,2}:\d{2}$')]

df_filtered['Hour'] = df_filtered['Time'].str.split(':').str[0].astype(int)
football_match_start_time = 15  # 15:00
football_match_end_time = 23  # 23:00

df_filtered['Football_Match_Time'] = df_filtered.apply(lambda row: 1 if row['Month'] in [8,9,10,11,12,1,2,3,4,5] and row['Day_of_Week'] in [1,2,3,4,5,6,7] and football_match_start_time <= row['Hour'] <= football_match_end_time else 0, axis=1)

df_filtered.to_csv('updated_dataset_with_football.csv', index=False)

df_filtered.tail(20)
#df_filtered['Football_Match'].unique()


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,Football_Match_Time
1504130,306950,604084,3,1,1,06/10/2014,2,07:30,Dual carriageway,70,...,Raining with high winds,Wet/Damp,2,2014,464677,7,1,Highway,10,0
1504131,319519,566948,2,2,3,27/09/2014,7,14:30,Single carriageway,30,...,Fine without high winds,Dry,2,2014,464678,14,0,Slow,9,0
1504132,313707,580203,3,6,1,14/10/2014,3,15:20,Dual carriageway,70,...,Fine without high winds,Dry,2,2014,464679,15,0,Highway,10,1
1504133,316784,568053,3,2,1,25/10/2014,7,07:45,Single carriageway,60,...,Raining without high winds,Wet/Damp,2,2014,464680,7,0,Highway,10,0
1504134,310347,594369,3,1,1,28/10/2014,3,11:14,Dual carriageway,70,...,Raining with high winds,Wet/Damp,2,2014,464681,11,0,Highway,10,0
1504135,319657,566553,2,2,1,05/11/2014,4,15:07,Single carriageway,30,...,Other,Wet/Damp,2,2014,464682,15,0,Slow,11,1
1504136,312209,580958,3,2,3,07/11/2014,6,08:50,Single carriageway,60,...,Fine without high winds,Wet/Damp,2,2014,464683,8,0,Highway,11,0
1504137,319487,566721,3,2,1,05/06/2014,5,15:00,Roundabout,30,...,Fine without high winds,Dry,2,2014,464684,15,0,Slow,6,0
1504138,310318,590421,2,3,1,17/11/2014,2,16:40,Single carriageway,60,...,Fine without high winds,Dry,2,2014,464685,16,1,Highway,11,1
1504139,311812,580747,3,1,1,17/11/2014,2,13:10,Single carriageway,60,...,Fine without high winds,Dry,2,2014,464686,13,0,Highway,11,0


In [125]:
df_filtered = df_filtered.dropna(subset=['Time'])
df_filtered = df_filtered[df_filtered['Time'].str.match(r'^\d{1,2}:\d{2}$')]
df_filtered['Hour'] = df_filtered['Time'].str.split(':').str[0].astype(int)

football_match_days = [6,7]  # 6 for Saturday, 7 for Sunday

football_match_start_time = 15 
football_match_end_time = 23  

df_filtered['Football_Match'] = df_filtered.apply(lambda row: 1 if row['Month'] in [8,9,10,11,12,1,2,3,4,5] and row['Day_of_Week'] in [6,7] and football_match_start_time <= row['Hour'] <= football_match_end_time else 0, axis=1)


df_filtered.to_csv('updated_dataset_with_football.csv', index=False)

df_filtered.tail(20)
#df_filtered['Football_Match'].unique()


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Road_Surface_Conditions,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,Football_Match_Time,Football_Match
1504130,306950,604084,3,1,1,06/10/2014,2,07:30,Dual carriageway,70,...,Wet/Damp,2,2014,464677,7,1,Highway,10,0,0
1504131,319519,566948,2,2,3,27/09/2014,7,14:30,Single carriageway,30,...,Dry,2,2014,464678,14,0,Slow,9,0,0
1504132,313707,580203,3,6,1,14/10/2014,3,15:20,Dual carriageway,70,...,Dry,2,2014,464679,15,0,Highway,10,1,0
1504133,316784,568053,3,2,1,25/10/2014,7,07:45,Single carriageway,60,...,Wet/Damp,2,2014,464680,7,0,Highway,10,0,0
1504134,310347,594369,3,1,1,28/10/2014,3,11:14,Dual carriageway,70,...,Wet/Damp,2,2014,464681,11,0,Highway,10,0,0
1504135,319657,566553,2,2,1,05/11/2014,4,15:07,Single carriageway,30,...,Wet/Damp,2,2014,464682,15,0,Slow,11,1,0
1504136,312209,580958,3,2,3,07/11/2014,6,08:50,Single carriageway,60,...,Wet/Damp,2,2014,464683,8,0,Highway,11,0,0
1504137,319487,566721,3,2,1,05/06/2014,5,15:00,Roundabout,30,...,Dry,2,2014,464684,15,0,Slow,6,0,0
1504138,310318,590421,2,3,1,17/11/2014,2,16:40,Single carriageway,60,...,Dry,2,2014,464685,16,1,Highway,11,1,0
1504139,311812,580747,3,1,1,17/11/2014,2,13:10,Single carriageway,60,...,Dry,2,2014,464686,13,0,Highway,11,0,0


In [126]:

df_filtered = df_filtered.dropna(subset=['Time'])


df_filtered = df_filtered[df_filtered['Time'].str.match(r'^\d{1,2}:\d{2}$')]
df_filtered['Hour'] = df_filtered['Time'].str.split(':').str[0].astype(int)


df_filtered.head(5)


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Road_Surface_Conditions,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,Football_Match_Time,Football_Match
0,525680,178240,2,1,1,04/01/2005,3,17:42,Single carriageway,30,...,Wet/Damp,1,2005,NaN,17,1,Slow,1,1,0
1,524170,181650,3,1,1,05/01/2005,4,17:36,Dual carriageway,30,...,Dry,1,2005,NaN,17,1,Slow,1,1,0
2,524520,182240,3,2,1,06/01/2005,5,00:15,Single carriageway,30,...,Dry,1,2005,NaN,0,0,Slow,1,0,0
3,526900,177530,3,1,1,07/01/2005,6,10:35,Single carriageway,30,...,Dry,1,2005,NaN,10,0,Slow,1,0,0
4,528060,179040,3,1,1,10/01/2005,2,21:13,Single carriageway,30,...,Wet/Damp,1,2005,NaN,21,0,Slow,1,1,0


In [127]:
unique_values = df['Light_Conditions'].unique()

print( unique_values)

['Daylight: Street light present'
 'Darkness: Street lights present and lit'
 'Darkness: Street lighting unknown'
 'Darkness: Street lights present but unlit'
 'Darkeness: No street lighting']


In [128]:
light_condition_map = {
    'Daylight: Street light present': 1,
    'Darkness: Street lights present and lit': 2,
    'Darkness: Street lighting unknown': 3,
    'Darkness: Street lights present but unlit': 4,
    'Darkness: No street lighting': 4
}

df_filtered['light_group'] = df_filtered['Light_Conditions'].map(light_condition_map)

print(df_filtered[['Light_Conditions', 'light_group']].head())
df_filtered.head(5)

                          Light_Conditions  light_group
0           Daylight: Street light present          1.0
1  Darkness: Street lights present and lit          2.0
2  Darkness: Street lights present and lit          2.0
3           Daylight: Street light present          1.0
4        Darkness: Street lighting unknown          3.0


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,Football_Match_Time,Football_Match,light_group
0,525680,178240,2,1,1,04/01/2005,3,17:42,Single carriageway,30,...,1,2005,NaN,17,1,Slow,1,1,0,1.0
1,524170,181650,3,1,1,05/01/2005,4,17:36,Dual carriageway,30,...,1,2005,NaN,17,1,Slow,1,1,0,2.0
2,524520,182240,3,2,1,06/01/2005,5,00:15,Single carriageway,30,...,1,2005,NaN,0,0,Slow,1,0,0,2.0
3,526900,177530,3,1,1,07/01/2005,6,10:35,Single carriageway,30,...,1,2005,NaN,10,0,Slow,1,0,0,1.0
4,528060,179040,3,1,1,10/01/2005,2,21:13,Single carriageway,30,...,1,2005,NaN,21,0,Slow,1,1,0,3.0


In [129]:
football_match_filter = df_filtered['Football_Match_Time'] == 1
dry_road_filter = df_filtered['Road_Surface_Conditions'] == 'Dry'
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Fine without high winds', 'Fine with high winds'])
light_group_filter = df_filtered['light_group'].isin([1,2])
speed_classification_filter = df_filtered['Speed_Classification'] == 'Slow'

q3updated = df_filtered[football_match_filter & dry_road_filter & Weather_Conditions_filter & light_group_filter & speed_classification_filter]

#q1 = q1.drop(columns=['Accident_Severity', 'Day_of_Week', 'Time', 'Local_Authority_(District)', 
#                     'Local_Authority_(Highway)', '1st_Road_Class', '1st_Road_Number', 
#                      'Special_Conditions_at_Site', 'Carriageway_Hazards', 'Year', 'Hour', 
#                     '2nd_Road_Class', '2nd_Road_Number', 'Junction_Detail', 
#                     'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities','Date'])


q3updated.head(5)

print("Number of rows in q2:", len(q3updated))
q3updated.head(5)
q3updated.to_csv('q3updated.csv', index=False)

Number of rows in q2: 260454


In [130]:
df_filtered = df_filtered.dropna(subset=['Date'])

df_filtered['Month'] = df_filtered['Date'].str.split('/').str[1].astype(int)
print("Unique values in 'Month':", df_filtered['Month'].unique())

df_filtered.tail(5)


Unique values in 'Month': [ 1  2  3  4  5  6  7  8  9 10 11 12]


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,Football_Match_Time,Football_Match,light_group
1504145,310037,597647,2,2,1,07/12/2014,1,22:20,Dual carriageway,70,...,2,2014,464692,22,0,Highway,12,1,0,NaN
1504146,321509,574063,3,2,2,11/12/2014,5,06:40,Dual carriageway,70,...,2,2014,464693,6,0,Highway,12,0,0,NaN
1504147,321337,566365,3,1,1,09/12/2014,3,02:00,Single carriageway,40,...,2,2014,464694,2,0,Non-Highway,12,0,0,2.0
1504148,323869,566853,2,3,2,17/12/2014,4,06:55,Single carriageway,60,...,2,2014,464695,6,0,Highway,12,0,0,NaN
1504149,314072,579971,3,2,2,24/12/2014,4,15:00,Single carriageway,60,...,2,2014,464696,15,0,Highway,12,1,0,1.0


In [131]:
football_match_filter = df_filtered['Football_Match_Time'] == 1
dry_road_filter = df_filtered['Road_Surface_Conditions'] == 'Dry'
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Fine without high winds', 'Fine with high winds'])
light_group_filter = df_filtered['light_group'].isin([1,2])
speed_classification_filter = df_filtered['Speed_Classification'] == 'Slow'

q3updated = df_filtered[football_match_filter & dry_road_filter & Weather_Conditions_filter & light_group_filter & speed_classification_filter]

#q1 = q1.drop(columns=['Accident_Severity', 'Day_of_Week', 'Time', 'Local_Authority_(District)', 
#                     'Local_Authority_(Highway)', '1st_Road_Class', '1st_Road_Number', 
#                      'Special_Conditions_at_Site', 'Carriageway_Hazards', 'Year', 'Hour', 
#                     '2nd_Road_Class', '2nd_Road_Number', 'Junction_Detail', 
#                     'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities','Date'])


q3updated.head(5)


print("Number of rows in q2:", len(q3updated))
q3updated.head(5)
#q3updated.to_csv('q3updated2.csv', index=False)
# Group data by Year and Month, then sum Football_Match_Time for each group
# Group data by Year and Month, then sum Football_Match_Time for each group
grouped_data = q3updated.groupby(['Year', 'Month'])['Football_Match_Time'].sum().reset_index()

filtered_data = q3updated[q3updated['Football_Match'] == 0].groupby(['Year', 'Month'])['Football_Match_Time'].sum().reset_index()

filtered_data.to_csv('non_football_match_time_by_year_monthjjjj.csv', index=False)
for _, row in filtered_data.iterrows():
    print(f"Year: {row['Year']}, Month: {row['Month']}, Total Football Match Time: {row['Football_Match_Time']}")



Number of rows in q2: 260454
Year: 2005, Month: 1, Total Football Match Time: 1651
Year: 2005, Month: 2, Total Football Match Time: 1719
Year: 2005, Month: 3, Total Football Match Time: 2552
Year: 2005, Month: 4, Total Football Match Time: 2660
Year: 2005, Month: 5, Total Football Match Time: 3551
Year: 2005, Month: 8, Total Football Match Time: 3285
Year: 2005, Month: 9, Total Football Match Time: 3038
Year: 2005, Month: 10, Total Football Match Time: 2535
Year: 2005, Month: 11, Total Football Match Time: 1974
Year: 2005, Month: 12, Total Football Match Time: 1497
Year: 2006, Month: 1, Total Football Match Time: 1836
Year: 2006, Month: 2, Total Football Match Time: 1785
Year: 2006, Month: 3, Total Football Match Time: 1861
Year: 2006, Month: 4, Total Football Match Time: 2363
Year: 2006, Month: 5, Total Football Match Time: 2856
Year: 2006, Month: 8, Total Football Match Time: 2709
Year: 2006, Month: 9, Total Football Match Time: 2776
Year: 2006, Month: 10, Total Football Match Time: 

In [132]:
urban_area_filter = df_filtered['Urban_or_Rural_Area'] == "1"
dry_road_filter = df_filtered['Road_Surface_Conditions'] == 'Dry'
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Fine without high winds', 'Fine with high winds'])


q7 = df_filtered[ urban_area_filter & dry_road_filter & Weather_Conditions_filter ]




q7.head(50)

#q7.to_csv('q7updated.csv', index=False)


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,Football_Match_Time,Football_Match,light_group
1,524170,181650,3,1,1,05/01/2005,4,17:36,Dual carriageway,30,...,1,2005,NaN,17,1,Slow,1,1,0,2.0
2,524520,182240,3,2,1,06/01/2005,5,00:15,Single carriageway,30,...,1,2005,NaN,0,0,Slow,1,0,0,2.0
3,526900,177530,3,1,1,07/01/2005,6,10:35,Single carriageway,30,...,1,2005,NaN,10,0,Slow,1,0,0,1.0
6,524220,180830,3,2,1,13/01/2005,5,20:40,Single carriageway,30,...,1,2005,NaN,20,0,Slow,1,1,0,2.0
7,525890,179710,3,1,2,14/01/2005,6,17:35,Dual carriageway,30,...,1,2005,NaN,17,0,Slow,1,1,1,1.0
8,527350,177650,3,2,2,15/01/2005,7,22:43,Single carriageway,30,...,1,2005,NaN,22,0,Slow,1,1,1,2.0
9,524550,180810,3,2,5,15/01/2005,7,16:00,Single carriageway,30,...,1,2005,NaN,16,0,Slow,1,1,1,1.0
10,526240,178900,3,1,1,16/01/2005,1,00:42,Single carriageway,30,...,1,2005,NaN,0,0,Slow,1,0,0,2.0
14,526700,178970,3,1,2,18/01/2005,3,11:15,Dual carriageway,30,...,1,2005,NaN,11,0,Slow,1,0,0,1.0
15,526460,177460,3,1,1,18/01/2005,3,10:50,Single carriageway,30,...,1,2005,NaN,10,0,Slow,1,0,0,1.0


In [133]:
Junction_control_map = {
    'Automatic traffic signal': 1,
    'Giveway or uncontrolled': 2,
    'Stop Sign': 3,
    'Authorised person': 4
}

df_filtered['Junction_group'] = df_filtered['Junction_Control'].map(Junction_control_map)

print(df_filtered[['Junction_Control', 'Junction_group']].head())

df_filtered.head(5)


           Junction_Control  Junction_group
0                       NaN             NaN
1  Automatic traffic signal             1.0
2                       NaN             NaN
3                       NaN             NaN
4                       NaN             NaN


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,Football_Match_Time,Football_Match,light_group,Junction_group
0,525680,178240,2,1,1,04/01/2005,3,17:42,Single carriageway,30,...,2005,NaN,17,1,Slow,1,1,0,1.0,NaN
1,524170,181650,3,1,1,05/01/2005,4,17:36,Dual carriageway,30,...,2005,NaN,17,1,Slow,1,1,0,2.0,1.0
2,524520,182240,3,2,1,06/01/2005,5,00:15,Single carriageway,30,...,2005,NaN,0,0,Slow,1,0,0,2.0,NaN
3,526900,177530,3,1,1,07/01/2005,6,10:35,Single carriageway,30,...,2005,NaN,10,0,Slow,1,0,0,1.0,NaN
4,528060,179040,3,1,1,10/01/2005,2,21:13,Single carriageway,30,...,2005,NaN,21,0,Slow,1,1,0,3.0,NaN


In [134]:
# Apply all filters directly to the DataFrame
q1 = df_filtered[
    (df_filtered['Rush_Hour'] == 1) &
    (df_filtered['Urban_or_Rural_Area'] == '1') &
    (df_filtered['Road_Surface_Conditions'] == 'Dry') &
    (df_filtered['Weather_Conditions'].isin(['Fine without high winds', 'Fine with high winds'])) &
    (df_filtered['light_group'].isin([1, 2])) &
    (df_filtered['Speed_Classification'] == 'Highway')
]

q1 = q1.dropna(subset=['Junction_Control'])

q1 = q1.drop(columns=['Football_Match_Time', 'Football_Match'])


print("Number of rows in q1:", len(q1))

q1.tail(10)


Number of rows in q1: 2228


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Road_Surface_Conditions,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,light_group,Junction_group
1498554,337074,672601,3,2,1,17/06/2014,3,07:30,Roundabout,60,...,Dry,1,2014,459101,7,1,Highway,6,1.0,2.0
1498635,333003,663327,3,3,2,30/06/2014,2,09:00,Single carriageway,60,...,Dry,1,2014,459182,9,1,Highway,6,1.0,2.0
1499944,281542,689747,3,2,1,01/07/2014,3,07:05,Roundabout,60,...,Dry,1,2014,460491,7,1,Highway,7,1.0,2.0
1500443,256625,664334,3,3,1,10/04/2014,5,17:40,Dual carriageway,70,...,Dry,1,2014,460990,17,1,Highway,4,1.0,2.0
1501131,252698,673548,3,3,2,11/03/2014,3,08:33,Single carriageway,60,...,Dry,1,2014,461678,8,1,Highway,3,1.0,2.0
1501161,254877,664266,3,2,2,07/04/2014,2,09:50,Slip road,70,...,Dry,1,2014,461708,9,1,Highway,4,1.0,1.0
1501385,252514,654999,3,2,2,10/07/2014,5,07:00,Single carriageway,60,...,Dry,1,2014,461932,7,1,Highway,7,1.0,2.0
1502047,246823,673058,3,2,2,10/09/2014,4,16:25,Dual carriageway,70,...,Dry,1,2014,462594,16,1,Highway,9,1.0,2.0
1502341,273390,662733,2,2,1,08/06/2014,1,18:15,Dual carriageway,70,...,Dry,1,2014,462888,18,1,Highway,6,1.0,2.0
1502457,273460,662720,3,2,1,17/12/2014,4,17:10,Dual carriageway,70,...,Dry,1,2014,463004,17,1,Highway,12,2.0,2.0


In [135]:
df_filtered.to_csv('df_filtered_output.csv', index=False)

In [136]:
rush_hour_filter = df_filtered['Rush_Hour'] == 1
urban_area_filter = df_filtered['Urban_or_Rural_Area'] == '1'
dry_road_filter = df_filtered['Road_Surface_Conditions'] == 'Dry'
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Fine without high winds', 'Fine with high winds'])
light_group_filter = df_filtered['light_group'].isin([1,2])
speed_classification_filter = df_filtered['Speed_Classification'] == 'Highway'

q2 = df_filtered[rush_hour_filter & urban_area_filter & dry_road_filter & Weather_Conditions_filter & light_group_filter & speed_classification_filter]


q2 = q2.drop(columns=['Football_Match_Time', 'Football_Match'])

q2.head(50)

print("Number of rows in q2:", len(q2))
q2.head(100)

Number of rows in q2: 5019


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Road_Surface_Conditions,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,light_group,Junction_group
2251,526930,181700,3,2,1,10/10/2005,2,07:40,Dual carriageway,60,...,Dry,1,2005,NaN,7,1,Highway,10,1.0,NaN
5653,537690,180580,2,2,1,15/03/2005,3,08:40,Dual carriageway,70,...,Dry,1,2005,NaN,8,1,Highway,3,1.0,NaN
6185,538280,181380,3,2,1,21/11/2005,2,16:45,Dual carriageway,70,...,Dry,1,2005,NaN,16,1,Highway,11,1.0,NaN
6666,537300,191090,3,2,1,07/06/2005,3,08:29,Dual carriageway,70,...,Dry,1,2005,NaN,8,1,Highway,6,1.0,NaN
8791,540590,181740,3,2,1,20/04/2005,4,18:30,Dual carriageway,70,...,Dry,1,2005,NaN,18,1,Highway,4,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49732,376480,360450,3,2,1,01/05/2005,1,16:30,Single carriageway,60,...,Dry,1,2005,NaN,16,1,Highway,5,1.0,NaN
49743,363750,390650,3,3,1,23/05/2005,2,07:47,Single carriageway,60,...,Dry,1,2005,NaN,7,1,Highway,5,1.0,NaN
49762,381130,354920,3,2,2,25/05/2005,4,18:30,Single carriageway,60,...,Dry,1,2005,NaN,18,1,Highway,5,1.0,2.0
49970,352990,379870,3,2,2,20/06/2005,2,07:05,Dual carriageway,60,...,Dry,1,2005,NaN,7,1,Highway,6,1.0,2.0


In [137]:
unique_values = df['Junction_Control'].unique()
print( unique_values)

[nan 'Automatic traffic signal' 'Giveway or uncontrolled' 'Stop Sign'
 'Authorised person']


In [138]:
football_match_filter = df_filtered['Football_Match_Time'] == 1
dry_road_filter = df_filtered['Road_Surface_Conditions'] == 'Dry'
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Fine without high winds', 'Fine with high winds'])
light_group_filter = df_filtered['light_group'].isin([1,2])
speed_classification_filter = df_filtered['Speed_Classification'] == 'Slow'

q3 = df_filtered[football_match_filter & dry_road_filter & Weather_Conditions_filter & light_group_filter & speed_classification_filter]
#q1 = q1.drop(columns=['Accident_Severity', 'Day_of_Week', 'Time', 'Local_Authority_(District)', 
#                     'Local_Authority_(Highway)', '1st_Road_Class', '1st_Road_Number', 
#                      'Special_Conditions_at_Site', 'Carriageway_Hazards', 'Year', 'Hour', 
#                     '2nd_Road_Class', '2nd_Road_Number', 'Junction_Detail', 
#                     'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities','Date'])

q3.head(5)
print("Number of rows in q2:", len(q3))
q3.head(5)

Number of rows in q2: 260454


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,Football_Match_Time,Football_Match,light_group,Junction_group
1,524170,181650,3,1,1,05/01/2005,4,17:36,Dual carriageway,30,...,2005,NaN,17,1,Slow,1,1,0,2.0,1.0
6,524220,180830,3,2,1,13/01/2005,5,20:40,Single carriageway,30,...,2005,NaN,20,0,Slow,1,1,0,2.0,2.0
7,525890,179710,3,1,2,14/01/2005,6,17:35,Dual carriageway,30,...,2005,NaN,17,0,Slow,1,1,1,1.0,NaN
8,527350,177650,3,2,2,15/01/2005,7,22:43,Single carriageway,30,...,2005,NaN,22,0,Slow,1,1,1,2.0,1.0
9,524550,180810,3,2,5,15/01/2005,7,16:00,Single carriageway,30,...,2005,NaN,16,0,Slow,1,1,1,1.0,2.0


In [139]:
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Raining without high winds',
 'Snowing without high winds', 'Raining with high winds','Fog or mist','Snowing with high winds'])
light_group_filter = df_filtered['light_group'].isin([3,2])


q4_badweather= df_filtered[  Weather_Conditions_filter & light_group_filter ]


q4_badweather = q4_badweather.drop(columns=['Football_Match_Time', 'Football_Match'])

q4_badweather.head(50)

print("Number of rows in q4:", len(q4_badweather))
q4_badweather.tail(15)

Number of rows in q4: 65663


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Road_Surface_Conditions,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,light_group,Junction_group
1503733,235065,626915,3,1,1,10/10/2014,6,07:20,Single carriageway,30,...,Wet/Damp,2,2014,464280,7,0,Slow,10,2.0,NaN
1503734,233614,621835,3,1,1,11/12/2014,5,16:30,Single carriageway,30,...,Wet/Damp,1,2014,464281,16,1,Slow,12,2.0,NaN
1503737,218481,597003,3,1,1,13/11/2014,5,03:50,Single carriageway,30,...,Wet/Damp,2,2014,464284,3,0,Slow,11,2.0,NaN
1503748,234419,620594,2,1,1,24/01/2014,6,17:20,Single carriageway,30,...,Wet/Damp,1,2014,464295,17,0,Slow,1,2.0,NaN
1503755,234332,621668,3,2,1,21/01/2014,3,18:40,Roundabout,30,...,Wet/Damp,1,2014,464302,18,1,Slow,1,2.0,2.0
1503815,236748,623330,3,2,1,22/12/2014,2,17:00,Single carriageway,50,...,Wet/Damp,2,2014,464362,17,1,Fast,12,2.0,NaN
1503829,232243,631196,3,1,1,21/11/2014,6,16:30,Single carriageway,30,...,Wet/Damp,1,2014,464376,16,0,Slow,11,2.0,1.0
1503905,209000,560865,3,2,2,09/12/2014,3,16:20,Single carriageway,60,...,Wet/Damp,2,2014,464452,16,1,Highway,12,2.0,NaN
1503918,276240,562259,3,1,1,25/04/2014,6,23:10,Single carriageway,30,...,Wet/Damp,2,2014,464465,23,0,Slow,4,2.0,2.0
1503945,310282,571749,3,1,1,03/01/2014,6,05:58,Single carriageway,60,...,Wet/Damp,2,2014,464492,5,0,Highway,1,2.0,2.0


In [140]:
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Fine without high winds','Fine with high winds'])
light_group_filter = df_filtered['light_group'].isin([3,2])


q4_goodweather= df_filtered[  Weather_Conditions_filter & light_group_filter ]


q4_goodweather = q4_goodweather.drop(columns=['Football_Match_Time', 'Football_Match'])

q4_goodweather.head(50)

print("Number of rows in q4:", len(q4_goodweather))
q4_goodweather.tail(15)

Number of rows in q4: 229101


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Road_Surface_Conditions,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,light_group,Junction_group
1503962,296217,575857,1,1,1,10/03/2014,2,00:01,Single carriageway,30,...,Dry,1,2014,464509,0,0,Slow,3,2.0,NaN
1503963,287863,595506,2,1,1,12/03/2014,4,23:12,Single carriageway,30,...,Dry,2,2014,464510,23,0,Slow,3,2.0,2.0
1504006,287540,584316,3,1,1,31/08/2014,1,10:30,Single carriageway,30,...,Dry,2,2014,464553,10,0,Slow,8,2.0,NaN
1504008,278618,609423,2,1,1,07/09/2014,1,20:30,Single carriageway,30,...,Dry,2,2014,464555,20,0,Slow,9,2.0,NaN
1504011,296350,577260,3,2,2,07/09/2014,1,22:20,Roundabout,60,...,Dry,1,2014,464558,22,0,Highway,9,2.0,2.0
1504021,296268,577250,3,2,1,13/10/2014,2,04:30,Single carriageway,60,...,Dry,1,2014,464568,4,0,Highway,10,2.0,NaN
1504022,295731,578094,3,1,1,22/10/2014,4,19:07,Single carriageway,30,...,Dry,1,2014,464569,19,0,Slow,10,2.0,NaN
1504025,297357,575646,2,2,1,04/11/2014,3,07:20,Dual carriageway,30,...,Wet/Damp,1,2014,464572,7,1,Slow,11,2.0,1.0
1504037,296145,577439,3,2,1,16/12/2014,3,07:15,Roundabout,30,...,Dry,1,2014,464584,7,1,Slow,12,2.0,2.0
1504038,303206,575243,2,2,1,18/12/2014,5,17:30,Single carriageway,50,...,Wet/Damp,2,2014,464585,17,1,Fast,12,2.0,NaN


In [141]:
urban_area_filter = df_filtered['Urban_or_Rural_Area'] == '1'
dry_road_filter = df_filtered['Road_Surface_Conditions'] == 'Dry'
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Fine without high winds', 'Fine with high winds'])
light_group_filter = df_filtered['light_group'].isin([1,2])
speed_classification_filter = df_filtered['Speed_Classification'] == 'Highway'

q5 = df_filtered[urban_area_filter & dry_road_filter & Weather_Conditions_filter & light_group_filter & speed_classification_filter]


q5 = q5.drop(columns=['Football_Match_Time', 'Football_Match'])

# Display the subset for verification
q5.head(50)

# Check the number of rows in the filtered subset
print("Number of rows in q12:", len(q5))
q5.tail(15)

Number of rows in q12: 15350


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Road_Surface_Conditions,Urban_or_Rural_Area,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,light_group,Junction_group
1503049,270524,660401,3,2,1,15/10/2014,4,14:05,Dual carriageway,70,...,Dry,1,2014,463596,14,0,Highway,10,1.0,NaN
1503056,277268,649393,3,2,2,22/09/2014,2,19:00,Dual carriageway,70,...,Dry,1,2014,463603,19,0,Highway,9,1.0,NaN
1503058,276916,649127,2,2,1,15/02/2014,7,21:06,Single carriageway,60,...,Dry,1,2014,463605,21,0,Highway,2,2.0,NaN
1503081,264393,655663,3,2,1,23/06/2014,2,19:53,Dual carriageway,60,...,Dry,1,2014,463628,19,0,Highway,6,1.0,2.0
1503267,261741,661900,3,1,1,23/05/2014,6,03:10,Dual carriageway,60,...,Dry,1,2014,463814,3,0,Highway,5,2.0,NaN
1503288,264677,661962,3,3,3,26/03/2014,4,12:30,Single carriageway,60,...,Dry,1,2014,463835,12,0,Highway,3,1.0,2.0
1503373,222379,643789,2,2,1,07/08/2014,5,19:00,Roundabout,60,...,Dry,1,2014,463920,19,0,Highway,8,1.0,2.0
1503442,219365,661598,3,3,1,24/08/2014,1,16:40,Single carriageway,60,...,Dry,1,2014,463989,16,1,Highway,8,1.0,NaN
1503572,244316,636410,3,2,2,26/08/2014,3,15:45,Single carriageway,60,...,Dry,1,2014,464119,15,0,Highway,8,1.0,NaN
1503659,236069,628726,3,3,2,02/12/2014,3,16:45,Dual carriageway,70,...,Dry,1,2014,464206,16,1,Highway,12,1.0,NaN


In [142]:
df_filtered.to_csv('df_filtered_output.csv', index=False)

In [143]:
q3.to_csv('q3NE.csv', index=False)

In [144]:
q4_badweather.to_csv('q4_badweatherNE.csv', index=False)

In [145]:
q4_goodweather.to_csv('q4_goodweatherNE.csv', index=False)

In [146]:
q5.to_csv('q5NE.csv', index=False)

In [147]:
q1.to_csv('q1NE.csv', index=False)

In [148]:
#q2.to_csv('q2updated.csv', index=False)
q2.count()

Location_Easting_OSGR      5019
Location_Northing_OSGR     5019
Accident_Severity          5019
Number_of_Vehicles         5019
Number_of_Casualties       5019
Date                       5019
Day_of_Week                5019
Time                       5019
Road_Type                  5019
Speed_limit                5019
Junction_Detail               0
Junction_Control           2228
Light_Conditions           5019
Weather_Conditions         5019
Road_Surface_Conditions    5019
Urban_or_Rural_Area        5019
Year                       5019
Unnamed: 0                 1338
Hour                       5019
Rush_Hour                  5019
Speed_Classification       5019
Month                      5019
light_group                5019
Junction_group             2228
dtype: int64

In [149]:
df_filtered.columns

Index(['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Accident_Severity',
       'Number_of_Vehicles', 'Number_of_Casualties', 'Date', 'Day_of_Week',
       'Time', 'Road_Type', 'Speed_limit', 'Junction_Detail',
       'Junction_Control', 'Light_Conditions', 'Weather_Conditions',
       'Road_Surface_Conditions', 'Urban_or_Rural_Area', 'Year', 'Unnamed: 0',
       'Hour', 'Rush_Hour', 'Speed_Classification', 'Month',
       'Football_Match_Time', 'Football_Match', 'light_group',
       'Junction_group'],
      dtype='object')

In [150]:
unique_values = df['Special_Conditions_at_Site'].unique()

# Print the unique values
print( unique_values)

[nan 'Ol or diesel' 'Roadworks' 'Auto traffic signal partly defective'
 'Road surface defective' 'Auto traffic singal out'
 'Permanent sign or marking defective or obscured' 'Mud']


In [151]:
unique_values = df['Carriageway_Hazards'].unique()

# Print the unique values
print( unique_values)

[nan 'Other object in carriageway'
 'Pedestrian in carriageway (not injured)'
 'Dislodged vehicle load in carriageway'
 'Involvement with previous accident' 'Any animal (except a ridden horse)']


In [152]:
print('Daylight: Street light present: 1, Darkness: Street lights present and lit: 2, Darkness: Street lighting unknown: 3, Darkness: Street lights present but unlit: 4, Darkness: No street lighting: 4')

Daylight: Street light present: 1, Darkness: Street lights present and lit: 2, Darkness: Street lighting unknown: 3, Darkness: Street lights present but unlit: 4, Darkness: No street lighting: 4


In [153]:
df.head()

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Year,Unnamed: 0
0,200501BS00001,525680,178240,-0.19117,51.489096,1,2,1,1,04/01/2005,...,Daylight: Street light present,Raining without high winds,Wet/Damp,NaN,NaN,1,Yes,E01002849,2005,NaN
1,200501BS00002,524170,181650,-0.211708,51.520075,1,3,1,1,05/01/2005,...,Darkness: Street lights present and lit,Fine without high winds,Dry,NaN,NaN,1,Yes,E01002909,2005,NaN
2,200501BS00003,524520,182240,-0.206458,51.525301,1,3,2,1,06/01/2005,...,Darkness: Street lights present and lit,Fine without high winds,Dry,NaN,NaN,1,Yes,E01002857,2005,NaN
3,200501BS00004,526900,177530,-0.173862,51.482442,1,3,1,1,07/01/2005,...,Daylight: Street light present,Fine without high winds,Dry,NaN,NaN,1,Yes,E01002840,2005,NaN
4,200501BS00005,528060,179040,-0.156618,51.495752,1,3,1,1,10/01/2005,...,Darkness: Street lighting unknown,Fine without high winds,Wet/Damp,NaN,NaN,1,Yes,E01002863,2005,NaN


In [154]:
q2.head(100)
unique_values = df['Road_Surface_Conditions'].unique()

print(unique_values)

['Wet/Damp' 'Dry' 'Frost/Ice' 'Snow' 'Flood (Over 3cm of water)' nan]


In [155]:
weather_condition_map = {
    'Fine without high winds': 0,
    'Fine with high winds': 1,
    'Raining without high winds': 2,
    'Raining with high winds': 3,
    'Snowing without high winds': 4,
    'Snowing with high winds':5
    
}


df_filtered['Weather_Group'] = df_filtered['Weather_Conditions'].map(weather_condition_map)


df_filtered[['Weather_Conditions', 'Weather_Group']].head()
df_filtered.tail(5)

,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,Football_Match_Time,Football_Match,light_group,Junction_group,Weather_Group
1504145,310037,597647,2,2,1,07/12/2014,1,22:20,Dual carriageway,70,...,464692,22,0,Highway,12,1,0,NaN,NaN,4.0
1504146,321509,574063,3,2,2,11/12/2014,5,06:40,Dual carriageway,70,...,464693,6,0,Highway,12,0,0,NaN,NaN,0.0
1504147,321337,566365,3,1,1,09/12/2014,3,02:00,Single carriageway,40,...,464694,2,0,Non-Highway,12,0,0,2.0,NaN,0.0
1504148,323869,566853,2,3,2,17/12/2014,4,06:55,Single carriageway,60,...,464695,6,0,Highway,12,0,0,NaN,2.0,2.0
1504149,314072,579971,3,2,2,24/12/2014,4,15:00,Single carriageway,60,...,464696,15,0,Highway,12,1,0,1.0,2.0,0.0


In [156]:
'Dry' 'Wet/Damp' 'Frost/Ice' 'Snow' 'Flood (Over 3cm of water)'
road_condition_map = {
    'Dry': 0,
    'Wet/Damp': 1,
    'Frost/Ice': 2,
    'Snow': 3,
    'Flood (Over 3cm of water)': 4,
    
    
}


df_filtered['road_condition_Group'] = df_filtered['Road_Surface_Conditions'].map(road_condition_map)


df_filtered[['Road_Surface_Conditions', 'road_condition_Group']].head()
df_filtered.tail(5)

,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Hour,Rush_Hour,Speed_Classification,Month,Football_Match_Time,Football_Match,light_group,Junction_group,Weather_Group,road_condition_Group
1504145,310037,597647,2,2,1,07/12/2014,1,22:20,Dual carriageway,70,...,22,0,Highway,12,1,0,NaN,NaN,4.0,3.0
1504146,321509,574063,3,2,2,11/12/2014,5,06:40,Dual carriageway,70,...,6,0,Highway,12,0,0,NaN,NaN,0.0,3.0
1504147,321337,566365,3,1,1,09/12/2014,3,02:00,Single carriageway,40,...,2,0,Non-Highway,12,0,0,2.0,NaN,0.0,2.0
1504148,323869,566853,2,3,2,17/12/2014,4,06:55,Single carriageway,60,...,6,0,Highway,12,0,0,NaN,2.0,2.0,1.0
1504149,314072,579971,3,2,2,24/12/2014,4,15:00,Single carriageway,60,...,15,0,Highway,12,1,0,1.0,2.0,0.0,1.0


In [157]:
urban_area_filter = df_filtered['Urban_or_Rural_Area']=='1'


q4_urban= df_filtered[ urban_area_filter ]

q4_urban = q4_urban.dropna(subset=['Weather_Group'])
q4_urban = q4_urban.dropna(subset=['road_condition_Group'])
q4_urban = q4_urban.drop(columns=['Football_Match_Time', 'Football_Match'])

q4_urban.head(50)

print("Number of rows in q4:", len(q4_urban))
q4_urban.tail(15)

Number of rows in q4: 929189


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,light_group,Junction_group,Weather_Group,road_condition_Group
1504004,296304,577164,3,2,1,29/08/2014,6,13:30,Roundabout,30,...,2014,464551,13,0,Slow,8,1.0,2.0,0.0,0.0
1504005,298234,575611,3,1,1,30/08/2014,7,15:35,Single carriageway,30,...,2014,464552,15,0,Slow,8,1.0,2.0,0.0,0.0
1504007,296098,577497,3,2,1,02/09/2014,3,10:30,Single carriageway,30,...,2014,464554,10,0,Slow,9,1.0,NaN,0.0,0.0
1504011,296350,577260,3,2,2,07/09/2014,1,22:20,Roundabout,60,...,2014,464558,22,0,Highway,9,2.0,2.0,1.0,0.0
1504012,295754,578066,3,1,1,18/09/2014,5,15:20,Single carriageway,30,...,2014,464559,15,0,Slow,9,1.0,NaN,0.0,0.0
1504013,297189,576345,3,2,1,18/09/2014,5,13:29,Single carriageway,30,...,2014,464560,13,0,Slow,9,1.0,2.0,0.0,0.0
1504014,297664,576157,3,3,1,24/09/2014,4,11:30,Single carriageway,30,...,2014,464561,11,0,Slow,9,1.0,2.0,0.0,0.0
1504017,297517,576177,3,2,2,27/09/2014,7,11:35,Single carriageway,30,...,2014,464564,11,0,Slow,9,1.0,2.0,0.0,0.0
1504021,296268,577250,3,2,1,13/10/2014,2,04:30,Single carriageway,60,...,2014,464568,4,0,Highway,10,2.0,NaN,0.0,0.0
1504022,295731,578094,3,1,1,22/10/2014,4,19:07,Single carriageway,30,...,2014,464569,19,0,Slow,10,2.0,NaN,0.0,0.0


In [158]:
urban_area_filter = df_filtered['Urban_or_Rural_Area']=='2'


q4_rural= df_filtered[ urban_area_filter ]

q4_rural = q4_rural.dropna(subset=['Weather_Group'])
q4_rural = q4_rural.dropna(subset=['road_condition_Group'])
q4_rural = q4_rural.drop(columns=['Football_Match_Time', 'Football_Match'])

q4_rural.head(50)

print("Number of rows in q4:", len(q4_rural))
q4_rural.tail(15)

Number of rows in q4: 504249


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,light_group,Junction_group,Weather_Group,road_condition_Group
1504132,313707,580203,3,6,1,14/10/2014,3,15:20,Dual carriageway,70,...,2014,464679,15,0,Highway,10,1.0,NaN,0.0,0.0
1504133,316784,568053,3,2,1,25/10/2014,7,07:45,Single carriageway,60,...,2014,464680,7,0,Highway,10,NaN,NaN,2.0,1.0
1504134,310347,594369,3,1,1,28/10/2014,3,11:14,Dual carriageway,70,...,2014,464681,11,0,Highway,10,1.0,NaN,3.0,1.0
1504136,312209,580958,3,2,3,07/11/2014,6,08:50,Single carriageway,60,...,2014,464683,8,0,Highway,11,1.0,2.0,0.0,1.0
1504137,319487,566721,3,2,1,05/06/2014,5,15:00,Roundabout,30,...,2014,464684,15,0,Slow,6,1.0,2.0,0.0,0.0
1504138,310318,590421,2,3,1,17/11/2014,2,16:40,Single carriageway,60,...,2014,464685,16,1,Highway,11,1.0,NaN,0.0,0.0
1504139,311812,580747,3,1,1,17/11/2014,2,13:10,Single carriageway,60,...,2014,464686,13,0,Highway,11,1.0,2.0,0.0,0.0
1504140,312649,583604,3,2,1,24/11/2014,2,06:50,Dual carriageway,70,...,2014,464687,6,0,Highway,11,NaN,NaN,0.0,1.0
1504141,316306,568290,1,1,3,25/11/2014,3,15:30,Single carriageway,60,...,2014,464688,15,0,Highway,11,1.0,NaN,0.0,1.0
1504144,307298,603520,3,2,1,08/12/2014,2,09:16,Dual carriageway,70,...,2014,464691,9,1,Highway,12,1.0,NaN,0.0,3.0


In [159]:
(q4_urban.to_csv('q4urban.csv', index=False))
q4_rural.to_csv('q4rural.csv', index=False)

In [160]:
dry_road_filter = df_filtered['Road_Surface_Conditions'] == 'Dry'
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Fine without high winds', 'Fine with high winds'])
light_group_filter = df_filtered['light_group'].isin([2,3,4])

q6 = df_filtered[ dry_road_filter & Weather_Conditions_filter & light_group_filter]


q6 = q6.drop(columns=['Football_Match_Time', 'Football_Match'])

q6.head(50)
#print("Number of rows in q6:", len(q6))


,Location_Easting_OSGR,Location_Northing_OSGR,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,...,Year,Unnamed: 0,Hour,Rush_Hour,Speed_Classification,Month,light_group,Junction_group,Weather_Group,road_condition_Group
1,524170,181650,3,1,1,05/01/2005,4,17:36,Dual carriageway,30,...,2005,NaN,17,1,Slow,1,2.0,1.0,0.0,0.0
2,524520,182240,3,2,1,06/01/2005,5,00:15,Single carriageway,30,...,2005,NaN,0,0,Slow,1,2.0,NaN,0.0,0.0
6,524220,180830,3,2,1,13/01/2005,5,20:40,Single carriageway,30,...,2005,NaN,20,0,Slow,1,2.0,2.0,0.0,0.0
8,527350,177650,3,2,2,15/01/2005,7,22:43,Single carriageway,30,...,2005,NaN,22,0,Slow,1,2.0,1.0,0.0,0.0
10,526240,178900,3,1,1,16/01/2005,1,00:42,Single carriageway,30,...,2005,NaN,0,0,Slow,1,2.0,1.0,0.0,0.0
16,524680,179450,2,2,1,20/01/2005,5,00:15,Single carriageway,30,...,2005,NaN,0,0,Slow,1,2.0,2.0,0.0,0.0
18,527810,178010,3,2,1,21/01/2005,6,21:16,Single carriageway,30,...,2005,NaN,21,0,Slow,1,2.0,NaN,0.0,0.0
19,526790,178980,2,1,1,08/01/2005,7,03:00,Single carriageway,30,...,2005,NaN,3,0,Slow,1,2.0,2.0,0.0,0.0
20,525940,178680,3,1,1,24/01/2005,2,21:45,Single carriageway,30,...,2005,NaN,21,0,Slow,1,2.0,NaN,0.0,0.0
21,524700,180880,3,2,1,24/01/2005,2,17:05,Single carriageway,30,...,2005,NaN,17,1,Slow,1,2.0,2.0,0.0,0.0


In [161]:
q6.to_csv('q6.csv', index=False)

In [162]:
dry_road_filter = df_filtered['Road_Surface_Conditions'] == 'Dry'
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Fine without high winds', 'Fine with high winds'])
light_group_filter = df_filtered['light_group'].isin([2])

q6group2 = df_filtered[ dry_road_filter & Weather_Conditions_filter & light_group_filter]


q6group2 = q6group2.drop(columns=['Football_Match_Time', 'Football_Match'])
q6group2.head(50)
q6group2.to_csv('q6group2.csv', index=False)



In [163]:
dry_road_filter = df_filtered['Road_Surface_Conditions'] == 'Dry'
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Fine without high winds', 'Fine with high winds'])
light_group_filter = df_filtered['light_group'].isin([4])

q6group4 = df_filtered[ dry_road_filter & Weather_Conditions_filter & light_group_filter]


q6group4 = q6group4.drop(columns=['Football_Match_Time', 'Football_Match'])
q6group4.head(50)
q6group4.to_csv('q6group4.csv', index=False)



In [164]:
dry_road_filter = df_filtered['Road_Surface_Conditions'] == 'Dry'
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Fine without high winds', 'Fine with high winds'])
light_group_filter = df_filtered['light_group'].isin([2,3,4])

q6 = df_filtered[ dry_road_filter & Weather_Conditions_filter & light_group_filter]


q6 = q6.drop(columns=['Football_Match_Time', 'Football_Match'])

q6.head(50)

#print("Number of rows in q6:", len(q6))

q6test = q6.tail(20500)

q6test.to_csv('q6testII.csv', index=False)
light_group_filter2 = q6test['light_group'].isin([2])

q6testgroup2 = q6test[ light_group_filter2]




q6testgroup2.to_csv('q6testgroup2II.csv', index=False)
light_group_filter3 = q6test['light_group'].isin([3])

q6testgroup3 = q6test[ light_group_filter3]




q6testgroup3.to_csv('q6testgroup3II.csv', index=False)
light_group_filter4 = q6test['light_group'].isin([4])

q6testgroup4 = q6test[ light_group_filter4]




q6testgroup4.to_csv('q6testgroup4II.csv', index=False)


In [165]:
q6train = q6.head(85250)

#output_csv = "q6traindata.csv"  # Replace with your desired output file name
#last_20_records.to_csv(output_csv, index=False)

In [166]:
light_group_filter = q6test['light_group'].isin([2])

q6testgroup2 = q6test[ light_group_filter]




q6testgroup2.to_csv('q6testgroup2.csv', index=False)



In [168]:
urban_area_filter = df_filtered['urban_area'] == "1"
dry_road_filter = df_filtered['Road_Surface_Conditions'] == 'Dry'
Weather_Conditions_filter = df_filtered['Weather_Conditions'].isin(['Fine without high winds', 'Fine with high winds'])


q7 = df_filtered[ urban_area_filter & dry_road_filter & Weather_Conditions_filter ]



q7.head(50)
#q7.count()
#q6group4.to_csv('q6group4.csv', index=False)


KeyError: 'urban_area'

In [176]:
import pandas as pd

filtered_data = pd.read_csv('q7updated.csv', dtype='unicode')

rush_hour_data = filtered_data[filtered_data['Rush_Hour'] == '1']

rush_hour_data['Urban_or_Rural_Area'] = rush_hour_data['Urban_or_Rural_Area'].apply(lambda x: int(x) if x.isdigit() else 0)

result = rush_hour_data.groupby(['Year', 'Month'])['Urban_or_Rural_Area'].sum().reset_index()
for _, row in result.iterrows():
    print({row['Urban_or_Rural_Area']})

result.to_csv('q7countrushhourcorrect.csv', index=False)

{1483}
{2070}
{1876}
{1288}
{1383}
{2003}
{2099}
{2755}
{2748}
{2257}
{2415}
{2594}
{1541}
{2037}
{1915}
{954}
{1618}
{1754}
{1925}
{2366}
{2561}
{2659}
{2036}
{2421}
{1430}
{2307}
{1791}
{1135}
{1341}
{2242}
{2437}
{2240}
{2062}
{2094}
{1936}
{2516}
{943}
{1856}
{1369}
{764}
{975}
{2173}
{2081}
{1884}
{2501}
{1852}
{1707}
{2453}
{795}
{1929}
{1441}
{546}
{959}
{2060}
{2069}
{2338}
{2421}
{2002}
{1763}
{2150}
{1183}
{1928}
{1827}
{884}
{1338}
{2367}
{2076}
{2343}
{2255}
{1994}
{1751}
{2313}
{1782}
{2041}
{1813}
{1017}
{1870}
{2954}
{1658}
{2737}
{1870}
{2287}
{2068}
{2616}
{968}
{1814}
{1437}
{1115}
{1242}
{1336}
{1803}
{1797}
{2039}
{2370}
{1802}
{1985}
{884}
{1693}
{1414}
{1214}
{1184}
{1951}
{1933}
{1741}
{2325}
{2422}
{1593}
{2408}


/var/folders/nn/px2pg7mx2dgdb26l6yfz_xr80000gn/T/ipykernel_3494/2973272565.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rush_hour_data['Urban_or_Rural_Area'] = rush_hour_data['Urban_or_Rural_Area'].apply(lambda x: int(x) if x.isdigit() else 0)


In [175]:
import pandas as pd
filtered_data = pd.read_csv('q7updated.csv', dtype='unicode')
rush_hour_data = filtered_data[filtered_data['Rush_Hour'] == '0']
rush_hour_data['Urban_or_Rural_Area'] = rush_hour_data['Urban_or_Rural_Area'].apply(lambda x: int(x) if x.isdigit() else 0)
result = rush_hour_data.groupby(['Year', 'Month'])['Urban_or_Rural_Area'].sum().reset_index()
for _, row in result.iterrows():
    print( {row['Urban_or_Rural_Area']})
    
result.to_csv('q7countrushhour.csv', index=False)


{4076}
{5291}
{4327}
{3760}
{3888}
{5103}
{5983}
{6514}
{6305}
{6489}
{6017}
{6045}
{4070}
{5006}
{4414}
{3269}
{4231}
{4422}
{5188}
{5152}
{6653}
{6771}
{5027}
{6100}
{3409}
{5462}
{4449}
{2847}
{3463}
{5403}
{5833}
{4935}
{5055}
{4778}
{5477}
{5442}
{2906}
{5053}
{2810}
{1958}
{3068}
{5055}
{4788}
{5477}
{5274}
{4818}
{4950}
{5421}
{1975}
{4801}
{3188}
{1343}
{2707}
{4450}
{5034}
{5315}
{5601}
{5307}
{4293}
{4828}
{2801}
{4970}
{3845}
{2673}
{3169}
{4917}
{5641}
{5035}
{4665}
{5134}
{4251}
{4911}
{4283}
{4749}
{4155}
{2692}
{4409}
{6850}
{4048}
{5859}
{4684}
{5251}
{5707}
{5856}
{2111}
{3599}
{3568}
{2572}
{3175}
{3094}
{3926}
{4166}
{4612}
{5216}
{4695}
{4117}
{2149}
{4202}
{3157}
{3043}
{2786}
{4640}
{4108}
{4419}
{4929}
{5037}
{4466}
{4872}


/var/folders/nn/px2pg7mx2dgdb26l6yfz_xr80000gn/T/ipykernel_3494/2530233585.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rush_hour_data['Urban_or_Rural_Area'] = rush_hour_data['Urban_or_Rural_Area'].apply(lambda x: int(x) if x.isdigit() else 0)
